In [1]:
# imports
import torch
from DataObjects.DataLoader import DataLoader
from Architectures.Transformer.Bert import BertFineTuner

from pathlib import Path
import numpy as np

/home/piotr/PycharmProjects/dl_project1/DeepLearning/Project_II/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-26 16:57:56,193	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-26 16:57:56,258	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# SETTING SEED
SEED = 42069
torch.manual_seed(SEED)
np.random.seed(0)

In [3]:
# set device to cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
DATA_PATH = Path("Data")

In [5]:
# prepare Data Loader files
train_dir = DATA_PATH / Path("MFCC_transformed", "training")
val_dir = DATA_PATH / Path("MFCC_transformed", "validation")
test_dir = DATA_PATH / Path("MFCC_transformed", "testing")

batch_size = 64

train_loader = DataLoader(
    data_dir=train_dir,
    data_type='mfcc',
    batch_size=batch_size,
    shuffle=True
)
val_loader = DataLoader(
    data_dir=val_dir,
    data_type='mfcc',
    batch_size=batch_size,
    shuffle=True
)

test_loader = DataLoader(
    data_dir=test_dir,
    data_type='mfcc',
    batch_size=batch_size,
    shuffle=True
)

# Inspect class mapping and dataset size
print("Classes:", train_loader.class_to_idx)
print("# training batches:", len(train_loader))

print("# validation batches:", len(val_loader))

print("# testing batches:", len(test_loader))

Classes: {'bed': 0, 'bird': 1, 'cat': 2, 'dog': 3, 'down': 4, 'eight': 5, 'five': 6, 'four': 7, 'go': 8, 'happy': 9, 'house': 10, 'left': 11, 'marvin': 12, 'nine': 13, 'no': 14, 'off': 15, 'on': 16, 'one': 17, 'right': 18, 'seven': 19, 'sheila': 20, 'silence': 21, 'six': 22, 'stop': 23, 'three': 24, 'tree': 25, 'two': 26, 'up': 27, 'wow': 28, 'yes': 29, 'zero': 30}
# training batches: 797
# validation batches: 108
# testing batches: 108


In [6]:
batch = next(iter(train_loader))
C, *rest = batch.data.shape[1:]
audio_dim = C

num_labels = len(train_loader.class_to_idx)

In [7]:
model = BertFineTuner(
    audio_dim=audio_dim,
    num_labels=num_labels,
    pretrained=True,
    model_name="gpt2",
    lr=5e-5
).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

You are using a model of type gpt2 to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', '

In [8]:
# training loop
model.train_architecture(train_loader = train_loader, val_loader=val_loader, epochs = 1)

Epoch 1/1 - train loss: 1.9749 - acc: 0.4150


Evaluating BERT: 100%|██████████| 108/108 [00:06<00:00, 17.28it/s]

Validation - loss: 0.8862 - acc: 0.7367 - f1: 0.7438


In [9]:
results = model.evaluate(test_loader)
print("Validation Summary:")
for k, v in results['summary'].items():
    print(f"  {k}: {v:.4f}")

Evaluating BERT: 100%|██████████| 108/108 [00:06<00:00, 17.20it/s]

Validation Summary:
  loss: 0.8754
  accuracy: 0.7382
  precision: 0.7656
  recall: 0.7464
  f1: 0.7439
